<a href="https://colab.research.google.com/github/cossiomanuel/Thesis_files/blob/main/Hystogram_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



```
# Tiene formato de código
```

# First trial with more memory.

In [1]:
#Load the modules
from glob import glob 
import numpy as np
import pandas as pd
import cv2,os
from sklearn.utils import shuffle

from tqdm import tqdm_notebook,trange
import matplotlib.pyplot as plt

import gc #garbage collection, we need to save all the RAM we can

In [2]:
# We are combining another piece of code to extract directly the files here
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

fid = drive.ListFile({'q':"title='dataset.zip'"}).GetList()[0]['id']
f = drive.CreateFile({'id': fid})
f.GetContentFile('dataset.zip')

In [3]:
!unzip dataset.zip

Se han truncado las últimas 5000 líneas del flujo de salida.
  inflating: train/9a85638e7d8a29bc02f0de3f937827ce98a79471.tif  
  inflating: __MACOSX/train/._9a85638e7d8a29bc02f0de3f937827ce98a79471.tif  
  inflating: train/898d82a450ac7adc097e956b9eaf7b4ae7286983.tif  
  inflating: __MACOSX/train/._898d82a450ac7adc097e956b9eaf7b4ae7286983.tif  
  inflating: train/b2bc0daec2c8c7a154bd0e17f2c79fb851650d73.tif  
  inflating: __MACOSX/train/._b2bc0daec2c8c7a154bd0e17f2c79fb851650d73.tif  
  inflating: train/1da035090ecbced39c2cf209e67d76c364d7588e.tif  
  inflating: __MACOSX/train/._1da035090ecbced39c2cf209e67d76c364d7588e.tif  
  inflating: train/7baf79b7d55c7552393071f4fd98b8f50f40c16a.tif  
  inflating: __MACOSX/train/._7baf79b7d55c7552393071f4fd98b8f50f40c16a.tif  
  inflating: train/1048fd5f36b296f7b76bb5ffbc41c3a19e0da1df.tif  
  inflating: __MACOSX/train/._1048fd5f36b296f7b76bb5ffbc41c3a19e0da1df.tif  
  inflating: train/50d24de9d3dbc78113a08e8dee9009819beffe10.tif  
  inflating: __

In [4]:
#set paths to training and test data
#path = "MACOSX/" #adapt this path, when running locally
train_path = 'train/'
test_path = 'test/'

df = pd.DataFrame({'path': glob(os.path.join(train_path,'*.tif'))}) # load the filenames
df['id'] = df.path.map(lambda x: x.split('/')[1].split(".")[0]) # keep only the file names in 'id'
labels = pd.read_csv("train_labels.csv") # read the provided labels
df = df.merge(labels, on = "id") # merge labels and filepaths
df.head(3) # print the first three entrys

,path,id,label
0,train/17214a72788b6ac24074e37c6565f90486a58073...,17214a72788b6ac24074e37c6565f90486a58073,0
1,train/fde411fd6bfa20a84dc0b5ba7343e602a3a0555b...,fde411fd6bfa20a84dc0b5ba7343e602a3a0555b,0
2,train/309e8935993a24cd253f1a1ab78bd19015638f09...,309e8935993a24cd253f1a1ab78bd19015638f09,0



Taken from complete-beginner-s-guide-eda-keras-lb-0-93

In [7]:
from tqdm import tqdm
from PIL import Image, ImageStat


def color_hist(input_image, nbins=32):
    ch1 = np.histogram(input_image[:,:,0], bins = nbins, range = (0, 256))[0] # [0] is because we need only the histogram, not bins edges
    ch2 = np.histogram(input_image[:,:,1], bins = nbins, range = (0, 256))[0]
    ch3 = np.histogram(input_image[:,:,2], bins = nbins, range = (0, 256))[0]
    return np.hstack((ch1, ch2, ch3))


def brightness(im_file):

  im = Image.open(im_file).convert('L')
  stat = ImageStat.Stat(im)
  return stat.mean[0]


def get_features(N, df, nbins=32):
    """ This functions loads N images using the data df
    """
    # allocate a numpy array for the images (N, 96x96px, 3 channels, values 0 - 255)
    X_hist = np.zeros([N, nbins*3], dtype=np.uint16)

    X_mb = np.zeros([N, 1], dtype=np.uint16)


    # convert the labels to a numpy array too
    y = np.squeeze(df[['label']].to_numpy())[0:N]

    '''Because as_matrix is deprecated I changed it for another df[['label]].to_numpy'''

    # read images one by one, tdqm notebook displays a progress bar
    for i, row in tqdm(df.iterrows(), total=N):
        if i == N:
            break
        X_hist[i] = color_hist(cv2.imread(row['path']))
        X_mb[i] = brightness(row['path'])

    return X_hist, X_mb, y

In [8]:
# Load 10k images
N=1000
X_hist, X_mb, y = get_features(N=N,df=df) 

 95%|█████████▌| 951/1000 [00:01<00:00, 692.13it/s]


In [9]:
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

X_scaler = StandardScaler().fit(X_hist)
scaled_X = X_scaler.transform(X_hist)
X_train, X_test, y_train, y_test = train_test_split(scaled_X, y, test_size=0.2, random_state=22)
svc = LinearSVC()
svc.fit(X_train, y_train)
test_acc = svc.score(X_test, y_test)
print(test_acc)

0.815


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [10]:
from sklearn.metrics import confusion_matrix
y_predict = svc.predict(X_test)
CM = confusion_matrix(y_test, y_predict)
print(CM)

[[113  17]
 [ 20  50]]


In [11]:
TP = CM[0][0]
TN = CM[1][1]
FN = CM[0][1]
FP = CM[1][0] 

error_rate = (FP + FN) / (FP + FN + TP + TN)
accuracy = (TP + TN) / (FP + FN + TP + TN)

sensitivity = TP / (TP + FN)
specificity = TN / (TN + FP)

precision = TP / (TP + FP)

false_pos_rate = FP / (TP + FN)
false_neg_rate = FN / (TN + FP)

F1_score = TP / (TP + ((FP + FN)/2))


summary = pd.DataFrame({'error_rate':[error_rate], 'accuracy':[accuracy], 'sensitivity':[sensitivity],
                        'specificity':[specificity], 'precision':[precision], 'false_pos_rate':[false_pos_rate],
                        'false_neg_rate':[false_neg_rate], 'F1_score':[F1_score]})

summary

,error_rate,accuracy,sensitivity,specificity,precision,false_pos_rate,false_neg_rate,F1_score
0,0.185,0.815,0.869231,0.714286,0.849624,0.153846,0.242857,0.859316
